In [5]:
import numpy as np
import trimesh
import matplotlib.pyplot as plt

In [21]:
obj_path = "../no_git_test_data/88-1/model_cleaned.obj"

tm_test = trimesh.load(obj_path)
# meshes = pyrender.Mesh.from_trimesh(tm_test)
# scene = pyrender.Scene()
# scene.add(meshes)

# # camera = pyrender.PerspectiveCamera(yfov=np.pi/3, aspectRatio=1)
# pyrender.Viewer(scene)

grid = np.indices((512, 512)).swapaxes(0, 2).reshape((-1, 2))
directions = np.concatenate([grid, np.ones((grid.shape[0], 1))], axis=1)
origins = np.zeros(directions.shape)
origins[:, -1] = -1

result = tm_test.ray.intersects_location(origins, directions, multiple_hits=True)

In [24]:
directions

array([[  0.,   0.,   1.],
       [  1.,   0.,   1.],
       [  2.,   0.,   1.],
       ...,
       [509., 511.,   1.],
       [510., 511.,   1.],
       [511., 511.,   1.]])

In [22]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(result[0])
o3d.io.write_point_cloud("../no_git_test_data/pcd/data.ply", pcd)

True

In [17]:
import open3d as o3d

eagle = o3d.data.EaglePointCloud()
pcd = o3d.io.read_point_cloud(eagle.path)

o3d.visualization.draw_geometries([pcd],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])


print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 796825 / 368254
[Open3D DEBUG] #   Got kernel density: 0.147 (s), 346.621 (MB) / 753.891 (MB) / 833 (MB)
